In [1]:
from sympy import *
import math
import numpy as np

In [2]:
# from Peter I. Kattan, MATLAB Guide to Finite Elements. An Interactive Approach, 2 Edition

In [3]:
def PlaneFrameElementLength(x1,y1,x2,y2):
    return math.sqrt((x2-x1)*(x2-x1) + (y2-y1)*(y2-y1))+10e-9

In [4]:
def PlaneFrameElementStiffness(E,A,I,L,theta):
    pi=math.pi 
    x = theta*pi/180
    C = math.cos(x)
    S = math.sin(x)
    w1 = A*C*C + 12*I*S*S/(L*L)
    w2 = A*S*S + 12*I*C*C/(L*L)
    w3 = (A-12*I/(L*L))*C*S
    w4 = 6*I*S/L
    w5 = 6*I*C/L
    
    return E/L*np.array([[w1, w3, -w4, -w1, -w3, -w4],[ w3, w2, w5, -w3, -w2, w5],
                        [-w4, w5, 4*I, w4, -w5, 2*I],[ -w1, -w3, w4, w1, w3, w4],
                        [-w3, -w2, -w5, w3, w2, -w5], [-w4, w5, 2*I, w4, -w5, 4*I]])  

In [5]:
def PlaneFrameAssemble(K,k,i,j):
    K[3*i,3*i] = K[3*i,3*i] + k[0,0]
    K[3*i,3*i+1] = K[3*i,3*i+1] + k[0,1]    
    K[3*i,3*i+2] = K[3*i,3*i+2] + k[0,2]
    K[3*i,3*j] = K[3*i,3*j] + k[0,3]
    K[3*i,3*j+1] = K[3*i,3*j+1] + k[0,4]
    K[3*i,3*j+2] = K[3*i,3*j+2] + k[0,5]
    K[3*i+1,3*i] = K[3*i+1,3*i] + k[1,0]
    K[3*i+1,3*i+1] = K[3*i+1,3*i+1] + k[1,1]
    K[3*i+1,3*i+2] = K[3*i+1,3*i+2] + k[1,2]
    K[3*i+1,3*j] = K[3*i+1,3*j] + k[1,3]
    K[3*i+1,3*j+1] = K[3*i+1,3*j+1] + k[1,4]
    K[3*i+1,3*j+2] = K[3*i+1,3*j+2] + k[1,5]
    K[3*i+2,3*i] = K[3*i+2,3*i] + k[2,0]
    K[3*i+2,3*i+1] = K[3*i+2,3*i+1] + k[2,1]
    K[3*i+2,3*i+2] = K[3*i+2,3*i+2] + k[2,2]
    K[3*i+2,3*j] = K[3*i+2,3*j] + k[2,3]
    K[3*i+2,3*j+1] = K[3*i,3*j+1] + k[2,4]
    K[3*i+2,3*j+2] = K[3*i+2,3*j+2] + k[2,5]
    K[3*j,3*i] = K[3*j,3*i] + k[3,0]
    K[3*j,3*i+1] = K[3*j,3*i+1] + k[3,1]
    K[3*j,3*i+2] = K[3*j,3*i+2] + k[3,2]
    K[3*j,3*j] = K[3*j,3*j] + k[3,3]
    K[3*j,3*j+1] = K[3*j,3*j+1] + k[3,4]
    K[3*j,3*j+2] = K[3*j,3*j+2] + k[3,5]   
    K[3*j+1,3*i] = K[3*j+1,3*i] + k[4,0]
    K[3*j+1,3*i+1] = K[3*j+1,3*i+1] + k[4,1]
    K[3*j+1,3*i+2] = K[3*j+1,3*i+2] + k[4,2]
    K[3*j+1,3*j] = K[3*j+1,3*j] + k[4,3]
    K[3*j+1,3*j+1] = K[3*j+1,3*j+1] + k[4,4]
    K[3*j+1,3*j+2] = K[3*j+1,3*j+2] + k[4,5]
    K[3*j+2,3*i] = K[3*j+2,3*i] + k[5,0]
    K[3*j+2,3*i+1] = K[3*j+2,3*i+1] + k[5,1]
    K[3*j+2,3*i+2] = K[3*j+2,3*i+2] + k[5,2]
    K[3*j+2,3*j] = K[3*j+2,3*j] + k[5,3]
    K[3*j+2,3*j+1] = K[3*j+2,3*j+1] + k[5,4]
    K[3*j+2,3*j+2] = K[3*j+2,3*j+2] + k[5,5]
    
    return K

In [6]:
def FEA_u(coord, elcon, bc_node, bc_val, global_force, I, A, E):
    K=np.zeros(shape=(3*np.max(elcon)+3,3*np.max(elcon)+3))
    pi=math.pi
    for el in elcon:
        L=PlaneFrameElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
        theta=math.atan((coord[el[1]][1]-coord[el[0]][1])/(coord[el[1]][0]-coord[el[0]][0]+1e-13))*180/pi
        k=PlaneFrameElementStiffness(E,A,I,L,theta)
        K=PlaneFrameAssemble(K,k,el[0],el[1])
    
    
    F = np.array(global_force)
    
    # https://github.com/CALFEM/calfem-matlab/blob/master/fem/solveq.m
    
    bc=np.array([bc_node, 
                bc_val]).T
    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))

    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    
    K[np.isnan(K)] = 0
    F[np.isnan(F)] = 0
    
    s=np.linalg.lstsq(K[fdof,:][:,fdof], (F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T, rcond=None)[0] 
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
    
#     Q=np.dot(K,d).T-F 
 
    return d

In [7]:
def internal_force(coord, elcon, u, I, A, E):
    ans=[]
    for el in elcon:
        L=PlaneFrameElementLength(coord[el[0]][0],coord[el[0]][1],coord[el[1]][0],coord[el[1]][1])
        theta=math.atan((coord[el[1]][1]-coord[el[0]][1])/(coord[el[1]][0]-coord[el[0]][0]+1e-13))*180/pi
        k=PlaneFrameElementStiffness(E,A,I,L,theta)
        ans.append(np.dot(k,u[3*el[0]:(3*el[1]+3)]))
    return ans               

In [8]:
I=400
A=30
E=20e6

In [9]:
coord=np.array([0,0,
               282.8, 282.8,
               682.8, 282.8]).reshape(3,2)

In [10]:
elcon=np.array([[0, 1],[1,2]])

In [11]:
bc_node=[0,1,6,7,8]

In [12]:
bc_val=[0,0,0,0,0]

In [13]:
global_force=np.array([0,0,0,
                      0,-100000,0,
                      0,0,0])

In [14]:
u = FEA_u(coord, elcon, bc_node, bc_val, global_force, I, A, E)

In [15]:
u

array([ 0.        ,  0.        , -0.0029696 ,  0.06697532, -0.20054933,
        0.00076369,  0.        ,  0.        ,  0.        ])

In [16]:
iforce = internal_force(coord, elcon, u, I, A, E)

In [17]:
iforce

[array([ 100462.98634395,   99928.2820395 , -150284.23387855,
        -100462.98634395,  -99928.2820395 ,    -930.14342314]),
 array([ 1.00462986e+05, -7.17179605e+01,  9.30143423e+02, -1.00462986e+05,
         7.17179605e+01, -2.96173276e+04])]

In [18]:
# TODO - the answer is not the same as in the book

### An illustrative example, p. 281

In [19]:
I=400
A=30
E=20e6

In [20]:
theta=45

In [21]:
x2 = theta*pi/180
C2 = math.cos(x2)
S2 = math.sin(x2)

In [22]:
coord=np.array([0,0,
                400*C2,400*S2,
                400*C2+400,400*S2]).reshape(3,2)

In [23]:
elcon=np.array([[0, 1],[1,2]])

In [24]:
bc_node=[0,1,2,6,7,8]

In [25]:
bc_val=[0,0,0,0,0,0]

In [26]:
global_force=np.array([0,0,0,
                      0,-100000,25000,
                      0,0,0])

In [27]:
u = FEA_u(coord, elcon, bc_node, bc_val, global_force, I, A, E)

In [28]:
u

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  6.61879976e-02,
       -1.98879326e-01,  1.77715795e-04,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])

In [29]:
iforce = internal_force(coord, elcon, u, I, A, E)

In [30]:
iforce

[array([ 99281.99633909,  99754.99575004,  63337.9023003 , -99281.99633909,
        -99754.99575004,  70446.53409492]),
 array([ 99281.9963391 ,   -245.00424995, -45446.53409491, -99281.9963391 ,
           245.00424995, -52555.16588953])]

In [31]:
# TODO - the answer is not the same as in the book (for internal force)